***CREATING AND TRAINING THE FACE EMOTION RECOGNITION MODEL USING TENSORFLOW CNN***



---

# 1.Importing all the libraries needed

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys, os
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils

# 2.Getting Data
We will be using the dataset fer-2013 which is publically available on Kaggle. it has 48*48 pixels gray-scale images of faces along with their emotion labels.

This dataset contains 7 Emotions :- (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)


In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

df=pd.read_csv('DATA/fer2013.csv')

**pd.read_csv(…)** is a function provided by the Pandas library and returns a DataFrame object that contains all the data with helpful and an easy to use api.

In [ ]:
print(df.info())
print(df["Usage"].value_counts())

print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


This dataset contains 3 columns, emotion, pixels and Usage. Emotion column contains integer encoded emotions and pixels column
contains pixels in the form of a string seperated by spaces, and usage
tells if data is made for training or testing purpose.

# 3.Preparing Data
You see data is not in the right format. we need to pre-process the data. Here X_train, X_test contains pixels, and y_test, y_train contains emotions.

In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
width, height = 48, 48

At this stage X_train, X_test contains pixel’s number is in the form of a string, converting it into numbers is easy, we just need to typecast.

In [ ]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [ ]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

**test_y, train_y** contains 1D integer encoded labels, we need to connect them into categorical data for efficient training.
**num_classes = num_labels = 7** shows that we have 7 classes to classify.

# 4.Reshaping Data
You need to convert the data in the form of a 4d tensor (row_num, width, height, channel) for training purposes.

In [ ]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

Here 1 tells us that training data is in grayscale form, at this stage, we have successfully preprocessed our data into **X_train, X_test, train_y, test_y**.

# 5.Building Facial Emotion Detection Model using CNN
Designing the CNN model for emotion detection using functional API. We are creating blocks using Conv2D layer, Batch-Normalization, Max-Pooling2D, Dropout, Flatten, and then stacking them together and at the end-use Dense Layer for output. 

Building the model using functional API gives more flexibility.

In [ ]:
model = Sequential()

### 1st 2D convolution layer (la convolution spatiale sur les images).

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))


#### model.add(BatchNormalization())
#le pooling layer
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

### 2nd 2D convolution layer 

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

#### model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

### 3th 2D convolution layer 
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
#le pooling layer
#### model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())


### fully connected layer ( FC)

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
#ajouter un layer costumé
model.add(Dense(num_labels, activation='softmax'))

model.summary()


# 6.Compiling the Facial Emotion Detection Model
Compiling model using **'Adam'** optimizer.

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# 7.Training the Facial Emotion Detection Model
To train the model you need to write the following line of code.

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, train_y,
          callbacks=[es_callback],
          batch_size=batch_size,
          epochs=25,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/25
449/449 [==============================] - 744s 2s/step - loss: 1.7100 - accuracy: 0.3016 - val_loss: 1.5300 - val_accuracy: 0.3904
Epoch 2/25
449/449 [==============================] - 893s 2s/step - loss: 1.4982 - accuracy: 0.4138 - val_loss: 1.4332 - val_accuracy: 0.4299
Epoch 3/25
449/449 [==============================] - 1025s 2s/step - loss: 1.3973 - accuracy: 0.4548 - val_loss: 1.3010 - val_accuracy: 0.5021
Epoch 4/25
449/449 [==============================] - 941s 2s/step - loss: 1.3351 - accuracy: 0.4824 - val_loss: 1.2692 - val_accuracy: 0.5191
Epoch 5/25
449/449 [==============================] - 948s 2s/step - loss: 1.2920 - accuracy: 0.5019 - val_loss: 1.2559 - val_accuracy: 0.5160
Epoch 6/25
449/449 [==============================] - 851s 2s/step - loss: 1.2537 - accuracy: 0.5190 - val_loss: 1.2134 - val_accuracy: 0.5316
Epoch 7/25
449/449 [==============================] - 770s 2s/step - loss: 1.2298 - accuracy: 0.5254 - val_loss: 1.2140 - val_accuracy: 0.535

# 8.Save the Model
Saving our model’s architecture into JSON and model’s weight into .h5.

In [ ]:
fer_json = model.to_json()
with open("DATA/fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("DATA/fer.h5")